The code to generate the volatility smiles from the MJD, BSWS, and Heston Models. Also includes some code on normal mixture diffusion however could not generate a volatility smile therefore was not included in main report.

In [6]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from matplotlib.lines import Line2D
from scipy.special import erfinv, erf
from scipy.special import iv #modified bessel function
from scipy import integrate
from scipy.stats import norm
import matplotlib.colors as mcolors

In [7]:
def GBM(mu,sigma,T,s0):
    t= np.linspace(0,T,100)
    step=T/len(t)
    stock = s0
    for i in range(len(t)):
      z = np.random.normal(0,1)
      stock = stock * np.exp((mu- 0.5 * sigma**2)*step + sigma*np.sqrt(step)*z)
    return stock

#CTMC
def GBM_WS(mu,sigmas,T,s0,Q):
    t = np.linspace(0, T, 100) 
    step = T/len(t)
    stock = s0
    states = list(range(len(Q)))
    current = np.random.choice(states,p=[1,0]) 
    sigma = sigmas[current]
    z = np.random.exponential(1 / abs(Q[current][current]))
    length = 0
    for i in range(len(t)):
        length += step
        if length >= z:
            length = 0
            p = Q[current] / abs(Q[current][current]) 
            probs = [pr if pr >= 0 else 0 for pr in p]
            current = np.random.choice(states, p=probs)
            sigma = sigmas[current]
            z = np.random.exponential(1 / abs(Q[current][current]))
        eta = np.random.normal(0, 1)
        stock = stock * np.exp((mu- 0.5 * sigma**2)*step + sigma*np.sqrt(step)*eta)
    return stock
#heston
def Stochastic_Volatility(mu,sig0,T,s0,delta,theta,k):
    sigma = sig0
    t= np.linspace(0,T,100)
    step=T/len(t)
    stock = s0
    rho=0.6
    for i in range(len(t)):
      Z = np.random.multivariate_normal(np.array([0,0]), cov = np.array([[1,rho],[rho,1]]))
      stock = stock * np.exp((mu- 0.5 * sigma**2)*step + sigma*np.sqrt(step)*Z[0])
      sigma = sigma - delta*(sigma - theta)*step +k*np.sqrt(step) *Z[1]
      sigma = np.abs(sigma)
    return stock

#Normal Mixture Diffusion
def NMD(mu,sigave,spread,T,s0):
    t= np.linspace(0,T,100)
    step=T/len(t)
    stock = s0
    for i in range(len(t)):
      z = np.random.normal(0,1)
      sigma = np.random.normal(sigave,spread)
      stock = stock * np.exp((mu- 0.5 * sigma**2)*step + sigma*np.sqrt(step)*z)
    return stock

#Merton Jump Diffusion
def GBM_MJD(mu,sigma,T,s0,lambda_jump,mu_jump,sigma_jump):
    t= np.linspace(0,T,100)
    step=T/len(t)
    stock = s0
   
    m = lambda_jump * mu_jump*-0.5
    for i in range(len(t)-1):
        #sign = np.random.choice([-1,1])
        z = np.random.normal(0, 1)  
        y = np.random.normal(mu_jump, sigma_jump)  
        Nt = np.random.poisson(lambda_jump * step) 
        stock = stock * np.exp((mu - 0.5 * sigma**2 - m) * step + sigma * np.sqrt(step) * z) * np.exp(Nt * y)
        
    
    return stock

#pricing CTMC
def phi(x,S0,sigmas,r,t,T):
    v = (sigmas[0]**2-sigmas[1]**2)*t + sigmas[1]**2 *T
    m = np.log(S0)+(r*T- (1/2 *v))
    return norm.pdf(x,m,np.sqrt(v))

def f0(Q,T,t):
    lam0 = abs(Q[0][0])
    lam1 = abs(Q[1][1])
    d=1
    sec = lam0*iv(0,2*(lam0*lam1*t*(T-t))**(1/2)) + ((lam0*lam1*t)/(T-t))**(1/2)*iv(1,2*(lam0*lam1*t*(T-t))**(1/2))
    out = np.exp(-lam0*T)*d + (np.exp(-lam1*(T-t)-lam0*t)* sec)
    return out

def f1(Q,T,t):
    lam0 = abs(Q[0][0])
    lam1 = abs(Q[1][1])
    d=0
    sec = (lam1*iv(0,2*(lam0*lam1*t*(T-t))**(1/2)))+((lam0*lam1*(T-t))/(t))**(1/2)*iv(1,2*(lam0*lam1*t*(T-t))**(1/2))
    out = np.exp(-lam1*T)*d + np.exp(-lam1*(T-t)-lam0*t) * sec
    return out

#pricing with switching
def BSWS_CALL(S,K,T,r,sigmas,Q):
    val,err = integrate.dblquad(lambda y,t: (y/(y+K)) * phi(np.log(y+K),S,sigmas,r,t,T)*f0(Q,T,t), 0, T, 0, np.inf)
    return val*np.exp(-r*T)

def BS_CALL(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + sigma**2 / 2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * norm.cdf(d1) - K * np.exp(-r*T)* norm.cdf(d2)

In [ ]:
conf_level = 0.97
c_p = 1-(1-conf_level)/2

S = 350
r = 0.3
mu=0.3
T = 1 
sigma=0.3

plt.axhline(sigma,linestyle=':',color='gray')

for k in range(250,450,10):
    M = 75
    V = np.zeros(M)
    for i in range(M):
        observed_price = np.zeros(10**3) #10000
        for s in range(10**3):
            observed_price[s] = max(GBM(mu,sigma,T,S)-k,0)
        
        op = np.mean(observed_price) * np.exp(-r*T)
        
        upper = 0.4
        lower = 0.2
        n=0
        while n<100:
            mid=(upper+lower)/2
            diff = op - BS_CALL(S, k , T, r, mid)
            if abs(diff) < 0.01:
                break
            elif diff>0:
                lower = mid
            elif diff<0:
                upper = mid
            n+=1
            
        V[i] = mid
    
    aM = np.mean(V)
    bM = np.std(V)
    c=np.sqrt(2)*erfinv(2*c_p-1)
    conf_lb = aM - c*bM/np.sqrt(M)
    conf_ub = aM + c*bM/np.sqrt(M)
    plt.vlines(k,conf_lb,conf_ub,color='black')
    plt.plot(k,aM,marker='h',markersize=3,color='#6969B3')

plt.xlabel('Strike Price')
plt.ylabel('Implied Volatility')
plt.title('Implied Volatility of Black Scholes')
plt.ylim([0.2,0.4])
plt.savefig("Images/IVBS.png", dpi=300)
plt.show()

In [ ]:
S = 350  
T = 0.25
r = 0.1
mu=r
sigmas = [0.1, 1]  

Q = np.array([[-2, 2], [4, -4]])

conf_level = 0.97
c_p = 1-(1-conf_level)/2

const = np.exp(-r*T)

for k in range(250,500,10):
    M = 300
    V = np.zeros(M)
    
        
    for i in range(M):
        op = 0
        observed_price = np.zeros(10**3)
        
        for s in range(10**3):
            observed_price[s] = max(GBM_WS(mu,sigmas,T,S,Q)-k,0)
        
        op = np.mean(observed_price) * const
        
        upper = 1
        lower = 0
        n=0
        while n<100:
            mid=(upper+lower)/2
            diff = op - BS_CALL(S, k , T, r, mid)
            if abs(diff) < 0.01:
                break
            elif diff>0:
                lower = mid
            elif diff<0:
                upper = mid
            n+=1
        
        
        V[i] = mid
    
    
    aM = np.mean(V)
    bM = np.std(V)
    c=np.sqrt(2)*erfinv(2*c_p-1)
    conf_lb = aM - c*bM/np.sqrt(M)
    conf_ub = aM + c*bM/np.sqrt(M)
    plt.vlines(k,conf_lb,conf_ub,color='black')
    plt.plot(k,aM,marker='h',markersize=3,color='blue')


plt.xlabel('Strike Price')
plt.ylabel('Implied Volatility')
plt.title('Implied Volitility of Black Scholes With Switching')
plt.savefig("Images/BSWSIV.png", dpi=300)
plt.show()

In [ ]:
S = 350
r = 0.1 
mu=0.1

sig0=0.5
T=1

delta =1#the rate it gets to sigma
theta = 0.5 #value it averages around
kon=1 #volatility of volatility

const = np.exp(-r*T)

conf_level = 0.97
c_p = 1-(1-conf_level)/2


for k in range(250,450,10):
    M = 300
    V = np.zeros(M)
    
    for i in range(M):
        observed_price = np.zeros(M)
        
        for s in range(M):
            observed_price[s] = max(Stochastic_Volatility(mu,sig0,T,S,delta,theta,kon)-k,0)
        
        op = np.mean(observed_price) * const
        
        upper = 1
        lower = 0
        n=0
        while n<100:
            mid=(upper+lower)/2
            diff = op - BS_CALL(S, k , T, r, mid)
            if abs(diff) < 0.1:
                break
            elif diff>0:
                lower = mid
            elif diff<0:
                upper = mid
            n+=1 
        V[i] = mid
    
    
    aM = np.mean(V)
    bM = np.std(V)
    c=np.sqrt(2)*erfinv(2*c_p-1)
    conf_lb = aM - c*bM/np.sqrt(M)
    conf_ub = aM + c*bM/np.sqrt(M)
    plt.vlines(k,conf_lb,conf_ub,color='black')
    plt.plot(k,aM,'.',color='brown')

plt.xlabel('Strike Price')
plt.ylabel('Implied Volatility')

plt.title('Implied Volitility of Heston Model')
plt.savefig("Images/StochCurve.png", dpi=300)
plt.show()

In [ ]:
#Normal Mixture Diffusion, attempted but could not get to work.
conf_level = 0.97
c_p = 1-(1-conf_level)/2

S = 350
r = 0.3
mu=0.3
T = 0.25 
sigma=0.3

const = np.exp(-r*T)
#plt.axhline(sigma,linestyle=':')


for k in range(200,500,10):
    M = 100
    V = np.zeros(M)
    observed_price = np.zeros(M)
    for i in range(M):
        for s in range(M):
            observed_price[s] = max(NMD(mu,0.3,0.5,T,S)-k,0)
        
        op = np.mean(observed_price) * const
        
        upper = 1
        lower = 0
        n=0
        while n<100:
            mid=(upper+lower)/2
            diff = op - BS_CALL(S, k , T, r, mid)
            if abs(diff) < 0.01:
                break
            elif diff>0:
                lower = mid
            elif diff<0:
                upper = mid
            n+=1
        
        
        V[i] = mid
    
    
    aM = np.mean(V)
    bM = np.std(V) 
    c=np.sqrt(2)*erfinv(2*c_p-1)
    conf_lb = aM - c*bM/np.sqrt(M)
    conf_ub = aM + c*bM/np.sqrt(M)
    plt.plot(k,aM,'x')
    plt.vlines(k,conf_lb,conf_ub)

plt.xlabel('Strike Price')
plt.ylabel('Implied Volatility')
plt.title('Implied Volatility of Black Scholes with NMD')
plt.show()

In [ ]:
S = 350
r = 0.1 
mu=r
T = 0.25

lambda_jump = 0.1  
mu_jump = 0.1  
sigma_jump = 0.5  
sigma=0.3

#lambda_jump = 0.2  
#mu_jump = 0.5  
#sigma_jump = 0.3   
#sigma=0.3

conf_level = 0.97
c_p = 1-(1-conf_level)/2
const = np.exp(-r*T)

for k in range(250,500,10):
    M = 100
    V = np.zeros(M)
    for i in range(M):
        observed_price = np.zeros(M)
        for s in range(M):
            observed_price[s] = max(GBM_MJD(mu,sigma,T,S,lambda_jump,mu_jump,sigma_jump)-k,0)
        
        op = np.mean(observed_price) * const
        upper = 1
        lower = 0
        n=0
        while n<100:
            mid=(upper+lower)/2
            diff = op - BS_CALL(S, k , T, r, mid)
            if abs(diff) < 0.01:
                break
            elif diff>0:
                lower = mid
            elif diff<0:
                upper = mid
            n+=1
        
        
        V[i] = mid
    
    
    aM = np.mean(V)
    bM = np.std(V)
    c=np.sqrt(2)*erfinv(2*c_p-1)
    conf_lb = aM - c*bM/np.sqrt(M)
    conf_ub = aM + c*bM/np.sqrt(M)
    plt.vlines(k,conf_lb,conf_ub,color='black')
    plt.plot(k,aM,marker='h',markersize=3,color='#6DA34D')

    
plt.xlabel('Strike Price')
plt.ylabel('Implied Volatility')
plt.title('Implied Volitility of Merton Jump Diffusion Model')
plt.show()